In [1]:
import pandas as pd
man_df = pd.read_csv('data/caesar_fr.csv', encoding='utf-8', sep=';')
man_df=man_df.sample(frac=1)
caesar_male = man_df[man_df['sex'] == 1]
caesar_male=caesar_male.loc[:, (caesar_male != 0).any()]

In [2]:
import pandas as pd
import numpy as np

def remplacer_nan_par_moyenne(df):
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            moyenne = df[col].mean(skipna=True)
            df[col] = df[col].fillna(moyenne)
    return df

caesar_malecaesar_malecaesar_male = remplacer_nan_par_moyenne(caesar_male)
caesar_male.head()

,taille,age,weight,sex,tour_de_cheville,longueur_du_bras,tour_de_poitrine,tour_de_poitrine_mesure_a_la_main,hauteur_de_poitrine,hauteur_d_entrejambe,...,hauteur_des_genoux,tour_du_cou,largeur_d_epaule,hauteur_des_epaules,largeur_des_epaules_a_l_horizontales,longueur_de_la_colonne_vertebrale_jusqu_au_poignet,tour_de_cuisse,tour_de_taille,hauteur_de_la_taille,longueur_d_avant_bras
3878,168.3004,25.3,65.798056,1.0,24.9936,59.5122,94.5896,NaN,123.901200,77.901800,...,46.710600,48.6918,37.007800,140.20800,44.9072,55.041800,57.5056,77.6986,99.4918,24.892000
666,180.7972,36.0,93.399129,1.0,28.9052,70.4088,99.2886,NaN,128.796549,81.218259,...,49.597886,48.8950,41.485936,145.86253,48.8950,58.414713,57.9882,92.6084,117.8052,26.407886
1004,175.9966,44.0,65.798056,1.0,23.7998,59.0042,91.1098,NaN,123.012200,78.587600,...,47.091600,44.5008,38.328600,142.06220,43.6880,56.845200,51.0032,83.4898,99.2886,24.904700
3781,174.3964,19.8,69.898527,1.0,25.9080,65.2018,92.7100,NaN,124.002800,80.492600,...,47.447200,46.5074,40.462200,140.58900,46.9900,57.835800,58.1914,81.2038,102.2096,25.463500
1778,177.6984,22.0,56.998371,1.0,25.1968,61.6966,84.3026,NaN,129.870200,85.496400,...,49.326800,45.9994,40.005000,146.41830,43.9928,59.029600,49.7078,68.3006,108.7882,26.085800


In [3]:
caesar_male["tour_de_cuisse_ratio"] = caesar_male["tour_de_cuisse"] / caesar_male["taille"]
caesar_male["tour_de_taille_ratio"] = caesar_male["tour_de_taille"] / caesar_male["taille"]
caesar_male["largeur_d_epaule_ratio"] = caesar_male["largeur_d_epaule"] / caesar_male["taille"]

In [4]:
def classifier_morphologie(row):
    # Extraction des mesures
    #tour_ventre = row['tour_de_ventre_maximal']
    tour_taille = row['tour_de_taille_ratio']
    forme_poitrine = row['largeur_d_epaule_ratio']
    tour_cuisse = row['tour_de_cuisse_ratio']
    
    
    # --- Catégorie VENTRE (basée sur le ratio ventre/taille) ---
    if tour_taille < 0.462864:
        ventre = "plat"
    elif 0.462864 <= tour_taille<0.536128:
        ventre = "moyen"
    else:
        ventre = "rond"
    
    # --- Catégorie TORSE (basée sur le tour de poitrine seul ou comparé à la taille) ---
    if forme_poitrine < 0.224841:
        torse = "fin"
    elif 0.224841 <=forme_poitrine < 0.242234 :
        torse = "moyen"
    else:
        torse = "large"
    
    # --- Catégorie CUISSES (ratio cuisse / hanches) ---
    if tour_cuisse < 0.312557:
        cuisses = "fines"
    elif  0.312557 <=tour_cuisse<0.350709 :
        cuisses = "moyennes"
    else:
        cuisses = "larges"
    
    return pd.Series([ventre, torse, cuisses], 
                     index=['categorie_ventre', 'categorie_torse', 'categorie_cuisses'])
    
# Remplace `df` par le nom de ton DataFrame
caesar_male[['categorie_ventre', 'categorie_torse', 'categorie_cuisses']] = caesar_male.apply(classifier_morphologie, axis=1)

In [5]:
# 1. Wrapper
class MultiTPOTWrapper:
    def __init__(self, models):
        self.models = models

    def fit(self, X, y=None):
        return self

    def predict(self, X):
        import numpy as np
        predictions = [m.predict(X) for m in self.models]
        return np.vstack(predictions).T

In [6]:
# 2. Pré-traitement
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import joblib


X_train=caesar_male[['taille', 'age', 'weight',
              'categorie_ventre', 'categorie_torse', 'categorie_cuisses']]

# Colonnes numériques et catégorielles
cols_num = ["taille","age","weight"]
cols_cat = ["categorie_ventre", "categorie_torse", "categorie_cuisses"]

# Pipeline pour les colonnes numériques : imputation par médiane + scaling
pipeline_num = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", RobustScaler())
])

# Pipeline pour les colonnes catégorielles : imputation par la valeur la plus fréquente + one-hot
pipeline_cat = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

# Pipeline globale de prétraitement
preprocessor = ColumnTransformer(transformers=[
    ("num", pipeline_num, cols_num),
    ("cat", pipeline_cat, cols_cat)
])

# Fit sur X_train (tu dois définir X_train comme DataFrame brut)
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', RobustScaler())]),
                                 ['taille', 'age', 'weight']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['categorie_ventre', 'categorie_torse',
                                  'categorie_cuisses'])])

In [7]:
# 3. Chargement modèles
import joblib
import os

dossier_pipelines = "pipelines_all_dataset"
models = []
for filename in sorted(os.listdir(dossier_pipelines)):
    if filename.endswith(".pkl"):
        path = os.path.join(dossier_pipelines, filename)
        models.append(joblib.load(path))
        print(f"✅ Modèle chargé : {filename}")

✅ Modèle chargé : pipeline_tpot_hauteur_d_entrejambe.pkl
✅ Modèle chargé : pipeline_tpot_hauteur_de_la_taille.pkl
✅ Modèle chargé : pipeline_tpot_hauteur_de_poitrine.pkl
✅ Modèle chargé : pipeline_tpot_hauteur_des_epaules.pkl
✅ Modèle chargé : pipeline_tpot_hauteur_des_genoux.pkl
✅ Modèle chargé : pipeline_tpot_hauteur_des_hanches.pkl
✅ Modèle chargé : pipeline_tpot_largeur_d_epaule.pkl
✅ Modèle chargé : pipeline_tpot_largeur_des_epaules_a_l_horizontales.pkl
✅ Modèle chargé : pipeline_tpot_longueur_d_avant_bras.pkl
✅ Modèle chargé : pipeline_tpot_longueur_de_la_colonne_vertebrale_jusqu_au_poignet.pkl
✅ Modèle chargé : pipeline_tpot_longueur_du_bras.pkl
✅ Modèle chargé : pipeline_tpot_tour_de_cheville.pkl
✅ Modèle chargé : pipeline_tpot_tour_de_cuisse.pkl
✅ Modèle chargé : pipeline_tpot_tour_de_poitrine.pkl
✅ Modèle chargé : pipeline_tpot_tour_de_taille.pkl
✅ Modèle chargé : pipeline_tpot_tour_du_cou.pkl


In [8]:
# 4. Pipeline complet
full_pipeline = Pipeline([
    ("preprocess", preprocessor),
    ("multi_model", MultiTPOTWrapper(models))
])

In [9]:
# 5. Sauvegarde
joblib.dump(full_pipeline, "full_pipeline_2.pkl")

# 6. Prédiction
pipeline_loaded = joblib.load("full_pipeline_2.pkl")

In [11]:
# ✅ Créer un wrapper à partir des pipelines chargés
wrapper = MultiTPOTWrapper(models)
new_data = pd.DataFrame([{
    "taille": 187,
    "age": 33,
    "weight": 80,
    "categorie_ventre": "moyen",
    "categorie_torse": "large",
    "categorie_cuisses": "medium"
    
}])
# ✅ Prédictions
preds = wrapper.predict(new_data)


c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-dataset\.venv\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\mbouke.besse\OneDrive - ESTIA\Documents\code-datase

In [12]:
# Suppose que tu as déjà ton pipeline chargé
# pipeline_loaded est de type Pipeline(steps=[("preprocess", ...), ("multi_model", ...)])

# Accès au préprocesseur uniquement
preprocessor = pipeline_loaded.named_steps["preprocess"]

# Applique le transformateur au new_data brut
X = preprocessor.transform(new_data)

# Si tu veux voir sous forme de DataFrame
import pandas as pd

# Récupère les noms des colonnes après transformation
# Cela fonctionne uniquement si le ColumnTransformer est simple
try:
    feature_names = preprocessor.get_feature_names_out()
    X_df = pd.DataFrame(X, columns=feature_names)
except:
    # fallback si les noms ne sont pas récupérables
    X_df = pd.DataFrame(X)

# Affiche les premières lignes
X_df.head()
X_df.columns

Index(['num__taille', 'num__age', 'num__weight', 'cat__categorie_ventre_moyen',
       'cat__categorie_ventre_plat', 'cat__categorie_ventre_rond',
       'cat__categorie_torse_fin', 'cat__categorie_torse_large',
       'cat__categorie_torse_moyen', 'cat__categorie_cuisses_fines',
       'cat__categorie_cuisses_larges', 'cat__categorie_cuisses_moyennes'],
      dtype='object')

In [13]:
# ✅ Extraire les noms des cibles
noms_variables = [f.replace("pipeline_tpot_", "").replace(".pkl", "") 
                  for f in sorted(os.listdir(dossier_pipelines)) if f.endswith(".pkl")]

# ✅ Associer les valeurs
predictions_dict = dict(zip(noms_variables, preds[0]))

# ✅ Affichage
print("📊 Prédictions :")
for k, v in predictions_dict.items():
    print(f" - {k} : {v:.2f}")


📊 Prédictions :
 - hauteur_d_entrejambe : 87.33
 - hauteur_de_la_taille : 111.03
 - hauteur_de_poitrine : 135.35
 - hauteur_des_epaules : 152.40
 - hauteur_des_genoux : 52.46
 - hauteur_des_hanches : 93.51
 - largeur_d_epaule : 43.23
 - largeur_des_epaules_a_l_horizontales : 48.46
 - longueur_d_avant_bras : 27.62
 - longueur_de_la_colonne_vertebrale_jusqu_au_poignet : 60.74
 - longueur_du_bras : 66.66
 - tour_de_cheville : 27.13
 - tour_de_cuisse : 59.55
 - tour_de_poitrine : 96.51
 - tour_de_taille : 83.59
 - tour_du_cou : 46.71
